In [1]:
import pandas as pd
import numpy as np

Reading New York data-- i have downloaded the data into csv file and added to watson studio

In [22]:
# The code was removed by Watson Studio for sharing.

,ZIP,Lat,Long
0,10001,40.750633,-73.997177
1,10002,40.715775,-73.986212
2,10003,40.731829,-73.989181
3,10004,40.688630,-74.018244
4,10005,40.706027,-74.008835


In [38]:

body = client_9931a902e795490ea510bfbf76e35bf1.get_object(Bucket='myassignment-donotdelete-pr-xoiu6dbqezzdve',Key='popdata1.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_ny_pop = pd.read_csv(body)
df_ny_pop.head()
#df_ny_pop.shape



,ZIP,Population
0,10001,"21,102"
1,10002,"81,410"
2,10003,"56,024"
3,10004,"3,089"
4,10005,"7,135"


In [34]:
df_ny_info=pd.merge(df_ny,df_ny_pop,on='ZIP')

In [46]:
#df_ny_info.head(5)
print(df_ny_info.columns)

Index(['ZIP', 'Lat', ' Long', 'Population'], dtype='object')


In [52]:
import folium
latitude=40.7999209
longitude=--73.9683102
map_NY = folium.Map(location=[latitude, longitude], zoom_start=12)
map_NY

# adding markers to map
for lat, lng, Population, ZIP in zip(df_ny_info['Lat'], df_ny_info[' Long'],df_ny_info['Population'], df_ny_info['ZIP']):
    label = '{}, {}'.format(ZIP, Population)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='Red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_NY)  
map_NY

In [53]:
limit = 500 # limit of number of venues returned by Foursquare API
radius = 5000 # define radius
CLIENT_ID = 'O0S5BRBBXQ3IMJZSZIFMVS2T31JXLRDFKBYJ3QXRL0OUCYOD'
CLIENT_SECRET = 'TJDWYDKOCFDH33UXN2ZQHWOBW05RZWSFMJ13QRHMJL03AOS0'
VERSION = '20190530'

In [67]:
# function to repeat the exploring process to all the neighborhoods in Newyork
import requests
LIMIT=500
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
      #  print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
       
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name'],
        v['venue']['categories'][0]['id']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category',
                            'Venue Id']
    
    return(nearby_venues)

In [68]:
ny_venues = getNearbyVenues(names=df_ny_info['ZIP'],
                                   latitudes=df_ny_info['Lat'],
                                   longitudes=df_ny_info[' Long']
                                  )

In [77]:
ny_venues
ny_restaurant=ny_venues[ny_venues['Venue Category'].str.contains('Restaurant')]
ny_restaurant.shape

(1140, 8)

In [71]:
# one hot encoding
ny_onehot = pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")


ny_onehot['Neighborhood'] = ny_venues['Neighborhood'] 


fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]

ny_onehot.head()


,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,...,Vietnamese Restaurant,Warehouse Store,Watch Shop,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,10001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [79]:
ny_grouped = ny_onehot.groupby('Neighborhood').mean().reset_index()


In [80]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

ny_grouped_clustering = ny_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ny_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 1, 1, 4, 0, 0, 0, 1, 1, 1], dtype=int32)

In [81]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [85]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ny_grouped['Neighborhood']

for ind in np.arange(ny_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ny_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(50)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,10001,Dance Studio,Café,Gym / Fitness Center,Pizza Place,Coffee Shop,Hotel,Music Venue,Burger Joint,French Restaurant,Food Court
1,10002,Coffee Shop,Cocktail Bar,Mexican Restaurant,Pizza Place,Bar,Chinese Restaurant,Café,Art Gallery,Sandwich Place,Ice Cream Shop
2,10003,Vegetarian / Vegan Restaurant,Dessert Shop,Yoga Studio,Japanese Restaurant,Ice Cream Shop,Coffee Shop,Grocery Store,Sushi Restaurant,Cosmetics Shop,Mediterranean Restaurant
3,10004,Food Truck,Art Gallery,Bike Rental / Bike Share,Garden,Campground,Beer Garden,Seafood Restaurant,Basketball Court,Baseball Field,Gym / Fitness Center
4,10005,Coffee Shop,American Restaurant,Cocktail Bar,Gym / Fitness Center,Gym,Steakhouse,Italian Restaurant,Hotel,Falafel Restaurant,Juice Bar
5,10006,Coffee Shop,Hotel,Pizza Place,Memorial Site,Clothing Store,Gym,Bar,Department Store,Italian Restaurant,Gym / Fitness Center
6,10007,Coffee Shop,Hotel,Italian Restaurant,Bakery,Gym,French Restaurant,Cocktail Bar,Bookstore,Sandwich Place,American Restaurant
7,10009,Cocktail Bar,Bar,Coffee Shop,Italian Restaurant,Pizza Place,Mexican Restaurant,Garden,Deli / Bodega,Wine Bar,Wine Shop
8,10010,Indian Restaurant,Italian Restaurant,Cocktail Bar,Mexican Restaurant,Thrift / Vintage Store,Gym,Hotel,Coffee Shop,Restaurant,American Restaurant
9,10011,Coffee Shop,American Restaurant,Ice Cream Shop,Bakery,Cosmetics Shop,Nightclub,Theater,Yoga Studio,Health & Beauty Service,Seafood Restaurant


In [90]:
ny_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,...,Vietnamese Restaurant,Warehouse Store,Watch Shop,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,10001,0.000000,0.00,0.00,0.000000,0.010000,0.00,0.00,0.000000,0.01,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
1,10002,0.000000,0.00,0.00,0.000000,0.030000,0.00,0.00,0.000000,0.00,...,0.020000,0.000000,0.00,0.000000,0.000000,0.010000,0.000000,0.000000,0.01,0.000000
2,10003,0.000000,0.00,0.00,0.000000,0.010000,0.00,0.00,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.020000,0.000000,0.00,0.040000
3,10004,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
4,10005,0.010000,0.00,0.00,0.000000,0.060000,0.00,0.00,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.010000,0.030000,0.000000,0.00,0.000000
5,10006,0.010000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.020000,0.000000,0.02,0.000000
6,10007,0.000000,0.00,0.00,0.000000,0.030000,0.01,0.00,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.010000,0.020000,0.010000,0.01,0.010000
7,10009,0.000000,0.00,0.00,0.000000,0.020000,0.01,0.00,0.010000,0.01,...,0.010000,0.000000,0.00,0.000000,0.000000,0.030000,0.030000,0.000000,0.00,0.000000
8,10010,0.000000,0.00,0.00,0.000000,0.030000,0.00,0.01,0.000000,0.00,...,0.010000,0.000000,0.00,0.000000,0.000000,0.000000,0.010000,0.000000,0.00,0.010000
9,10011,0.000000,0.00,0.00,0.000000,0.040000,0.01,0.00,0.010000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,0.020000


In [86]:
df_pop=df_ny_info.copy()
df_pop.head()

,ZIP,Lat,Long,Population
0,10001,40.750633,-73.997177,"21,102"
1,10002,40.715775,-73.986212,"81,410"
2,10003,40.731829,-73.989181,"56,024"
3,10004,40.688630,-74.018244,"3,089"
4,10005,40.706027,-74.008835,"7,135"


In [89]:
df_pop=df_pop.rename(columns={"ZIP":"Neighborhood"})
df_pop.head()

,Neighborhood,Lat,Long,Population
0,10001,40.750633,-73.997177,"21,102"
1,10002,40.715775,-73.986212,"81,410"
2,10003,40.731829,-73.989181,"56,024"
3,10004,40.688630,-74.018244,"3,089"
4,10005,40.706027,-74.008835,"7,135"


In [91]:
neighborhoods_venues_sorted=pd.merge(neighborhoods_venues_sorted,df_pop,on='Neighborhood')

In [92]:
neighborhoods_venues_sorted.head(5)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Lat,Long,Population
0,10001,Dance Studio,Café,Gym / Fitness Center,Pizza Place,Coffee Shop,Hotel,Music Venue,Burger Joint,French Restaurant,Food Court,40.750633,-73.997177,"21,102"
1,10002,Coffee Shop,Cocktail Bar,Mexican Restaurant,Pizza Place,Bar,Chinese Restaurant,Café,Art Gallery,Sandwich Place,Ice Cream Shop,40.715775,-73.986212,"81,410"
2,10003,Vegetarian / Vegan Restaurant,Dessert Shop,Yoga Studio,Japanese Restaurant,Ice Cream Shop,Coffee Shop,Grocery Store,Sushi Restaurant,Cosmetics Shop,Mediterranean Restaurant,40.731829,-73.989181,"56,024"
3,10004,Food Truck,Art Gallery,Bike Rental / Bike Share,Garden,Campground,Beer Garden,Seafood Restaurant,Basketball Court,Baseball Field,Gym / Fitness Center,40.688630,-74.018244,"3,089"
4,10005,Coffee Shop,American Restaurant,Cocktail Bar,Gym / Fitness Center,Gym,Steakhouse,Italian Restaurant,Hotel,Falafel Restaurant,Juice Bar,40.706027,-74.008835,"7,135"


In [119]:
neighborhoods_venues_=neighborhoods_venues_sorted[(neighborhoods_venues_sorted['1st Most Common Venue'].str.contains('Restaurant') | neighborhoods_venues_sorted['1st Most Common Venue'].str.contains('Food')) ]

In [122]:
neighborhoods_venues_.head(16)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Lat,Long,Population
2,10003,Vegetarian / Vegan Restaurant,Dessert Shop,Yoga Studio,Japanese Restaurant,Ice Cream Shop,Coffee Shop,Grocery Store,Sushi Restaurant,Cosmetics Shop,Mediterranean Restaurant,40.731829,-73.989181,"56,024"
3,10004,Food Truck,Art Gallery,Bike Rental / Bike Share,Garden,Campground,Beer Garden,Seafood Restaurant,Basketball Court,Baseball Field,Gym / Fitness Center,40.688630,-74.018244,"3,089"
8,10010,Indian Restaurant,Italian Restaurant,Cocktail Bar,Mexican Restaurant,Thrift / Vintage Store,Gym,Hotel,Coffee Shop,Restaurant,American Restaurant,40.739065,-73.982255,"31,834"
10,10012,Italian Restaurant,Clothing Store,Boutique,Shoe Store,Café,Sushi Restaurant,Ice Cream Shop,Gym,French Restaurant,Sporting Goods Shop,40.725581,-73.998078,"24,090"
12,10014,Italian Restaurant,Cosmetics Shop,Gastropub,New American Restaurant,Coffee Shop,Park,Men's Store,Japanese Restaurant,Theater,Wine Bar,40.734012,-74.006746,"31,959"
13,10016,Italian Restaurant,Hotel,Sushi Restaurant,Gym / Fitness Center,Coffee Shop,Bar,Bakery,Salon / Barbershop,Pizza Place,Spa,40.745224,-73.978297,"54,183"
18,10021,Italian Restaurant,Coffee Shop,Bakery,Burger Joint,Mexican Restaurant,Gym / Fitness Center,Gym,Gift Shop,Spanish Restaurant,Salon / Barbershop,40.769258,-73.958751,"43,631"
20,10023,Italian Restaurant,Theater,Coffee Shop,Concert Hall,French Restaurant,Gym,Sushi Restaurant,Juice Bar,Indie Movie Theater,Gym / Fitness Center,40.775921,-73.982607,"60,998"
22,10025,Indian Restaurant,Mexican Restaurant,Coffee Shop,Pizza Place,Deli / Bodega,Chinese Restaurant,Gym / Fitness Center,Grocery Store,Ice Cream Shop,Gastropub,40.798601,-73.966622,"94,600"
25,10028,Italian Restaurant,Bar,Pizza Place,Japanese Restaurant,Mexican Restaurant,Yoga Studio,Thai Restaurant,Bakery,Coffee Shop,Ice Cream Shop,40.776441,-73.953509,"45,141"
